In [1]:
import pandas as pd

In [2]:
import os

file_list = []

a = open("output.txt", "w")
for path, subdirs, files in os.walk(r'/Users/jennifertang/Desktop/CAPSTONE/9GB original dataset/Video_Dataset/bad_toe/1115_video'):
    for filename in files:
        f = os.path.join(path, filename)
        file_list.append(f)

In [3]:
file_list

['/Users/jennifertang/Desktop/CAPSTONE/9GB original dataset/Video_Dataset/bad_toe/1115_video/1003_squat_000165.mp4',
 '/Users/jennifertang/Desktop/CAPSTONE/9GB original dataset/Video_Dataset/bad_toe/1115_video/1113_squat_000019.mp4',
 '/Users/jennifertang/Desktop/CAPSTONE/9GB original dataset/Video_Dataset/bad_toe/1115_video/1003_squat_000159.mp4',
 '/Users/jennifertang/Desktop/CAPSTONE/9GB original dataset/Video_Dataset/bad_toe/1115_video/1018_squat_000216.mp4',
 '/Users/jennifertang/Desktop/CAPSTONE/9GB original dataset/Video_Dataset/bad_toe/1115_video/1113_squat_000025.mp4',
 '/Users/jennifertang/Desktop/CAPSTONE/9GB original dataset/Video_Dataset/bad_toe/1115_video/0918_squat_000079.mp4',
 '/Users/jennifertang/Desktop/CAPSTONE/9GB original dataset/Video_Dataset/bad_toe/1115_video/1018_squat_000014.mp4',
 '/Users/jennifertang/Desktop/CAPSTONE/9GB original dataset/Video_Dataset/bad_toe/1115_video/1025_squat_000208.mp4',
 '/Users/jennifertang/Desktop/CAPSTONE/9GB original dataset/Vide

In [4]:
len(file_list)

305

In [5]:
file_list[0]

'/Users/jennifertang/Desktop/CAPSTONE/9GB original dataset/Video_Dataset/bad_toe/1115_video/1003_squat_000165.mp4'

In [6]:
import cv2
import mediapipe as mp
from time import time, sleep

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose


In [7]:
file_list_revised = []
for i in file_list:
    file_list_revised.append(i.split('/'))

In [8]:
file_list_revised[0][-1][:-3]

'1003_squat_000165.'

In [9]:
for i in range(len(file_list)):

    l=[]
    cap = cv2.VideoCapture(file_list[i])
    timestep=0.01
    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                break

            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                #print(landmarks)

                l.append(landmarks)
                sleep(timestep)
            except:
                pass


            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style() 
                                      )               

            #cv2.imshow('Mediapipe Feed', cv2.flip(image, 1))

            if cv2.waitKey(10) & 0xFF == 27:
                break

        cap.release()
        cv2.destroyAllWindows()

        cv2.waitKey(1)   #add this for mac user
        
    dct = {}
    for j in range(0,99):
        dct[str(j)] = []
            
    for t in l:
        for b in range(33):
            dct[str(b*3)].append(t[b].x)
            dct[str(b*3+1)].append(t[b].y)
            dct[str(b*3+2)].append(t[b].z)
                
    cols=[]
    for bodypart in mp_pose.PoseLandmark:
        cols.append(str(bodypart).replace("PoseLandmark.", "")+"_X")
        cols.append(str(bodypart).replace("PoseLandmark.", "")+"_Y")
        cols.append(str(bodypart).replace("PoseLandmark.", "")+"_Z")
            
    df=pd.DataFrame(dct)
    df.columns=cols
        

    df.to_csv('/Users/jennifertang/Desktop/bad_toe_coordinates/'+file_list_revised[i][-1][:-3]+'csv')